# PyTorch Experiment Tracking drone detect system

## Datasets
I choosen pictures from:
- https://www.kaggle.com/datasets/umairatwork/manzoorumair-cloud-cover-dataset-muccd
- Open Images V7
- Cranfield Synthetic Drone Detection Dataset: https://huggingface.co/datasets/mazqtpopx/cranfield-synthetic-drone-detection

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install ultralytics

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 949.8/949.8 kB 36.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 86.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 81.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 57.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 42.6 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstall

In [3]:
import zipfile
from pathlib import Path

zip_path = '/content/drive/MyDrive/Colab Notebooks/data_yolo.zip'
# data_path = Path("data/")
image_path = ""

with zipfile.ZipFile(zip_path, "r") as zip_ref:
        print("Unzipping data...")
        zip_ref.extractall(image_path)

Unzipping data...


In [4]:
import yaml
base_path = Path('data')

yaml_content = {
    'path': '/content/data',
    'train': 'images/train',
    'val': 'images/val',
    'test': 'images/test',

    'nc': 6,
    'names': {
        0: 'airplane',
        1: 'balloon',
        2: 'bird',
        3: 'drone',
        4: 'helicopter',
        5: 'sky'
    }
}

with open('data.yaml', 'w') as f:
    yaml.dump(yaml_content, f, sort_keys=False, default_flow_style=False)

In [ ]:
from ultralytics import YOLO
import matplotlib.pyplot as plt
import numpy as np

def on_train_epoch_end(trainer):
    try:
        current_epoch = trainer.epoch + 1

        # Training loss
        train_loss = [x.detach().cpu().numpy() for x in trainer.loss_items]

        # Validation metrics
        if trainer.validator:
            map50 = trainer.validator.metrics.box.map50
            metrics_text = f"mAP50: {map50:.2f}"
        else:
            metrics_text = ""

        # Tworzenie wykresów
        plt.figure(figsize=(12, 6))

        # Wykres strat treningowych
        plt.subplot(1, 2, 1)
        plt.plot(train_loss, label='Train Loss')
        plt.title(f'Epoch {current_epoch} - Loss')
        plt.legend()

        # Wyświetlanie metryk jako tekst
        plt.subplot(1, 2, 2)
        plt.text(0.1, 0.5, metrics_text, fontsize=12)
        plt.axis('off')

        # Zapis i zamknięcie
        plt.tight_layout()
        plt.savefig(f'epoch_{current_epoch}_metrics.png')
        plt.close()

    except Exception as e:
        print(f"Błąd w callbacku: {str(e)}")

# Model initialisation with callbacks
model = YOLO('yolov8n.pt')
model.add_callback('on_train_epoch_end', on_train_epoch_end)

# Configuration of hyperparameters
hyp = {
    'optimizer': 'SGD',
    'lr0': 0.01,              # Początkowy współczynnik uczenia (learning rate)
    'lrf': 0.1,               # Końcowy współczynnik uczenia
    'momentum': 0.937,        # Momentum dla optymalizatora SGD - pomaga uniknąć minimów lokalnych
    'weight_decay': 0.0005,   # Regularyzacja L2 - zapobiega przeuczeniu poprzez karanie dużych wartości wag
    'warmup_epochs': 3.0,     # Liczba epok rozgrzewki - stopniowe zwiększanie learning rate na początku treningu
    'box': 7.5,               # Współczynnik straty dla lokalizacji bounding box (im wyższa wartość, tym większy nacisk na precyzję bboxów)
    'cls': 0.5,               # Współczynnik straty klasyfikacji (decyduje o wadze poprawnej klasyfikacji)
    'dfl': 1.5                # Współczynnik straty rozkładu (Distribution Focal Loss) - ważne dla małych obiektów
}

# Training
results = model.train(
    data='data.yaml',
    epochs=150,
    imgsz=640,
    batch=16,
    project='aerial_detection',
    name='exp3',
    patience=20,              # zatrzyma trening jeśli brak poprawy przez 20 epok
    augment=True,
    mixup=0.2,
    mosaic=0.5,
    degrees=10.0,
    translate=0.1,
    scale=0.5,
    shear=2.0,
    perspective=0.0005,
    flipud=0.3,
    fliplr=0.5,
    hsv_h=0.015,
    hsv_s=0.7,
    hsv_v=0.4,
    **hyp
)


Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


100%|██████████| 6.25M/6.25M [00:00<00:00, 318MB/s]


Ultralytics 8.3.97 🚀 Python-3.11.11 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: task=detect, mode=train, model=yolov8n.pt, data=data.yaml, epochs=150, time=None, patience=20, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=aerial_detection, name=exp3, exist_ok=False, pretrained=True, optimizer=SGD, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=True, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True, show_boxes=True, l

100%|██████████| 755k/755k [00:00<00:00, 72.8MB/s]


Overriding model.yaml nc=80 with nc=6

                   from  n    params  module                                       arguments                     
  0                  -1  1       464  ultralytics.nn.modules.conv.Conv             [3, 16, 3, 2]                 
  1                  -1  1      4672  ultralytics.nn.modules.conv.Conv             [16, 32, 3, 2]                
  2                  -1  1      7360  ultralytics.nn.modules.block.C2f             [32, 32, 1, True]             
  3                  -1  1     18560  ultralytics.nn.modules.conv.Conv             [32, 64, 3, 2]                
  4                  -1  2     49664  ultralytics.nn.modules.block.C2f             [64, 64, 2, True]             
  5                  -1  1     73984  ultralytics.nn.modules.conv.Conv             [64, 128, 3, 2]               
  6                  -1  2    197632  ultralytics.nn.modules.block.C2f             [128, 128, 2, True]           
  7                  -1  1    295424  ultralytics

...

     93/150      5.95G      1.343      1.147      1.362          1        640: 100%|██████████| 297/297 [02:26<00:00,  2.02it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 38/38 [00:15<00:00,  2.43it/s]


                   all       1187       4706      0.778      0.665      0.706      0.472

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


     94/150      5.96G      1.338      1.127      1.346         31        640: 100%|██████████| 297/297 [02:25<00:00,  2.04it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 38/38 [00:16<00:00,  2.34it/s]


                   all       1187       4706      0.807      0.668      0.717      0.477

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


     95/150      5.96G      1.314      1.107      1.337         65        640:  20%|█▉        | 58/297 [00:27<01:37,  2.46it/s]

In [ ]:
best_model = YOLO('aerial_detection/exp3/weights/best.pt')

# Evaluate on a test set
metrics = best_model.val(
    data='data.yaml',
    split='test'
)

print(f'mAP@50: {metrics.box.map50}')

In [ ]:
import shutil
import os

google_drive_path = '/content/drive/MyDrive/Colab Notebooks/YOLOv8_AP_results'
os.makedirs(google_drive_path, exist_ok=True)

results_path = '/content/aerial_detection'

shutil.copytree(results_path, os.path.join(google_drive_path, 'train_results'), dirs_exist_ok=True)

print(f"The results were saved to Google Drive in the folder: {google_drive_path}")